# IoT Challenge - Geolocalization

## Data exploration [Théo]

## Features Engineering [Mathieu]

### One-Hot (Teacher ++ / Mean / RSSI / etc.. )

### Usage of centroid

#### Standard centroid

#### Weighted centroid

## Outliers processing I [André]

## Model [Mathieu / Matyas / André]

### First raw predictions

### Outliers processing II

### Fine-tunning / Model selection

### Blending

In [ ]:
## Test pull request

## Test analysis [André] 